# requirimentos

In [ ]:
!pip install gcsfs datasets
!pip install --upgrade sentence-transformers
!pip install --upgrade transformers
!pip install unidecode
!python -m spacy download pt_core_news_md
!pip install --upgrade torch


# treino

In [ ]:
from datasets import Dataset
import pandas as pd
import torch
from transformers import (
    BertForSequenceClassification,
    BertTokenizer,
    Trainer,
    TrainingArguments,
    EarlyStoppingCallback
)
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.utils.class_weight import compute_class_weight
from sklearn.metrics import accuracy_score, f1_score

# Fixar semente para reprodução
torch.manual_seed(42)

# Carregar os dados
csv_path = "/content/dataset_transformado.csv"  # Atualize com o caminho correto
data = pd.read_csv(csv_path)

# Codificar as classes
label_encoder = LabelEncoder()
data["Label"] = label_encoder.fit_transform(data["Label"])

# Calcular pesos das classes
class_weights = compute_class_weight(
    class_weight="balanced",
    classes=data["Label"].unique(),
    y=data["Label"]
)
class_weights_tensor = torch.tensor(class_weights, dtype=torch.float)

# Dividir os dados em treino e teste
train_texts, eval_texts, train_labels, eval_labels = train_test_split(
    data["Texto"], data["Label"], test_size=0.1, random_state=42
)

# Tokenizar os textos
tokenizer = BertTokenizer.from_pretrained("neuralmind/bert-large-portuguese-cased")

train_encodings = tokenizer(list(train_texts), truncation=True, padding=True, max_length=128)
eval_encodings = tokenizer(list(eval_texts), truncation=True, padding=True, max_length=128)

# Criar datasets compatíveis com o Trainer
class ClassificationDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item["labels"] = torch.tensor(self.labels[idx])
        return item

train_dataset = ClassificationDataset(train_encodings, list(train_labels))
eval_dataset = ClassificationDataset(eval_encodings, list(eval_labels))

# Carregar o modelo para classificação
model = BertForSequenceClassification.from_pretrained(
    "neuralmind/bert-large-portuguese-cased",
    num_labels=len(label_encoder.classes_),  # Número de classes
)

# Configurar os argumentos de treinamento
training_args = TrainingArguments(
    output_dir="models/bert-ptbr-classification",
    num_train_epochs=4,  # Aumentado para explorar mais o treinamento
    per_device_train_batch_size=16,  # Reduzido para evitar problemas de memória
    per_device_eval_batch_size=32,
    warmup_steps=500,
    learning_rate=2e-5,  # Taxa de aprendizado ajustada
    weight_decay=0.01,
    logging_dir="/content/logs",
    logging_steps=10,
    evaluation_strategy="steps",
    eval_steps=452,  # Avaliações mais frequentes
    save_steps=452,
    save_total_limit=2,
    load_best_model_at_end=True,
    metric_for_best_model="accuracy",
    greater_is_better=True,
    fp16=True,
    run_name="bert-ptbr-classification",
)

# Definir a métrica de avaliação
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = torch.argmax(torch.tensor(logits), dim=1).numpy()
    return {
        "accuracy": accuracy_score(labels, predictions),
        "f1": f1_score(labels, predictions, average="weighted"),
    }

# Criar o Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
    callbacks=[EarlyStoppingCallback(early_stopping_patience=3)]  # Adicionado callback para early stopping
)

# Treinar o modelo
trainer.train()

# Salvar o modelo treinado
output_dir = f"models/bert-ptbr-classification-final"
model.save_pretrained(output_dir)
tokenizer.save_pretrained(output_dir)


In [ ]:
# Carregar o modelo e o tokenizer salvos
from transformers import BertForSequenceClassification, BertTokenizer
import torch

# Diretório onde o modelo foi salvo
output_dir = "models/bert-ptbr-classification-final"

model = BertForSequenceClassification.from_pretrained(output_dir)
tokenizer = BertTokenizer.from_pretrained(output_dir)

# Frase para teste
test_phrases = [

    
    "loja de reparo de celular perto"
]


# Tokenizar as frases de teste
inputs = tokenizer(test_phrases, return_tensors="pt", truncation=True, padding=True, max_length=128)

# Fazer a predição
model.eval()  # Colocar o modelo em modo de avaliação
with torch.no_grad():
    outputs = model(**inputs)
    logits = outputs.logits
    print(logits)
    predictions = torch.argmax(logits, dim=1).numpy()
    print(f' predicitions --->{predictions}')

# Decodificar os índices das classes previstas
predicted_classes = label_encoder.inverse_transform(predictions)

# Exibir as frases e suas respectivas previsões
for phrase, pred_class in zip(test_phrases, predicted_classes):
    print(f"Frase: '{phrase}' -> Classe prevista: '{pred_class} prob {predictions}'")
    pass


O gato subiu na árvore e miou alto.



In [ ]:


probabilities = torch.softmax(logits, dim=1)

values, indices = torch.topk(probabilities, 10, dim=1)
rounded_values = [round(value,2) for value in values[0].tolist()]

print("Probabilidades dos 10 maiores:",rounded_values )  
print("Índices dos 10 maiores:", label_encoder.inverse_transform(indices[0].tolist()))

